# Recommendation Systems
### Size uygun öneri verme sistemidir

## iki temel algoritması vardır;

###   *User-Based Collaborative Filtering
###  *Item-Based Collaborative Filtering

#

# User-Based Collaborative Filtering

### Kullanıcılar arasındaki benzerlik skarlarını bulmaya çalışır.
### Her bir kullanıcının satın aldığı veya izlediği/dinlediği/tükettiği materyali kullanıcı bazlı olarak bir matriste tutar. Bir kullanıcıya (x) benzer başka kullanıcılar bulur. X kullanıcısının henüz tüketmediği ancak benzer kullanıcılarının çoğunun tükettiği bir içerik varsa bunu X kullanıcısına önerir

## Dezavantajları;
### kullanıcı bazlı olduğu için maliyetlidir. İnsan sayısı içerik sayısından her zaman fazladır
### kullanıcı zevkleri değişkenlik gösterebilir

#

# Item-Based Collaborative Filtering

### Itemler arasındaki benzerlik skorlarını bulmaya çalışır.

### Kullanıcılar yerine Itemları baz alır. Itemlar arasındaki ilişki ve benzerlikle ilgilenir ve birbirine benzeye Itemları bulur. ( her iki filmi beğenenler aynı, demekki bu iki film benzer)

#

In [1]:
import numpy as np
import pandas as pd

In [2]:
columnNames = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('users.data', sep='\t', names = columnNames)
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [3]:
len(df)

100003

In [4]:
movieTitles = pd.read_csv('movie_id_titles.csv')
movieTitles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
len(movieTitles)

1682

In [6]:
df = pd.merge(df, movieTitles, on = 'item_id')
# iki tabloyu item_id'ye göre değiştiriyoruz
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


## Recommedation Sisteminin Kurulumu;

### Öncelikle exceldeki pivot tablo benzeri bir yapı kuruyoruz.
### Bu yapıya göre her satır bir kullanacı olacak (yani dataframe'in index'i user_id olacak)
### Sütunlarda film isimleri (title) olacak
### tablo içinde de rating değerli olacak şekilde bir dataframe oluşturuyoruz

In [7]:
moviemat = df.pivot_table(index = 'user_id' , columns = 'title' , values = 'rating')
moviemat.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
type(moviemat)

pandas.core.frame.DataFrame

## Amaç : Star wars filmine benzer film önerileri yapmak.

### Star Wars filminin ratinglerine bakıyoruz;

In [9]:
starwarsUserRating = moviemat['Star Wars (1977)']
starwarsUserRating.head(20)

user_id
0     5.0
1     5.0
2     5.0
3     NaN
4     5.0
5     4.0
6     4.0
7     5.0
8     5.0
9     5.0
10    5.0
11    NaN
12    4.0
13    5.0
14    5.0
15    5.0
16    NaN
17    NaN
18    4.0
19    NaN
Name: Star Wars (1977), dtype: float64

### corrwith() metodunu kullanarak Star Wars filmi ile korelasyonları hesaplatalım:

In [10]:
similarToStarwars = moviemat.corrwith(starwarsUserRating)
similarToStarwars

E:\Development\Anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
E:\Development\Anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


title
'Til There Was You (1997)                0.872872
1-900 (1994)                            -0.645497
101 Dalmatians (1996)                    0.211132
12 Angry Men (1957)                      0.184289
187 (1997)                               0.027398
                                           ...   
Young Guns II (1990)                     0.228615
Young Poisoner's Handbook, The (1995)   -0.007374
Zeus and Roxanne (1997)                  0.818182
unknown                                  0.723123
Á köldum klaka (Cold Fever) (1994)            NaN
Length: 1664, dtype: float64

In [11]:
type(similarToStarwars)

pandas.core.series.Series

### Bazı kayıtlarda boşluklar olduğu için hata veriyor. similarToStarwars bir seri, biz bunu corrStarwars isimli bir dataframe'e dönüştürelim ve NaN kayıtlarını temizleyip bakalım;

In [12]:
# Correlation verilerini sütun haline getiriyorum
corrStarwars = pd.DataFrame(similarToStarwars, columns = ['Correlation'])
#dropna ile NaN verileri silinir
corrStarwars.dropna(inplace = True)
corrStarwars.head(20)

,Correlation
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
2 Days in the Valley (1996),0.066654
"20,000 Leagues Under the Sea (1954)",0.289768
2001: A Space Odyssey (1968),0.230884
"39 Steps, The (1935)",0.106453


### Elde ettiğimiz dataframe'i sıralıyoruz ve Star Wars'a en yakın tavsiye edeceği filmlere bakıyoruz;

In [13]:
corrStarwars.sort_values('Correlation',ascending=False).head(10)

,Correlation
title,
Commandments (1997),1.0
Cosi (1996),1.0
No Escape (1994),1.0
Stripes (1981),1.0
Man of the Year (1995),1.0
Hollow Reed (1996),1.0
"Beans of Egypt, Maine, The (1994)",1.0
"Good Man in Africa, A (1994)",1.0
"Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)",1.0


## Görüldüğü üzere alakasız filmler çıktı. Bunun sebebi bu filmlerin az oy almış olmasıdır.
## Bunu düzeltmek için 100'den az oy alan filmleri eleyelim.

In [14]:
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [15]:
# df'yi 'title'a göre gruplar ve reating sütunun ortalamasını alır ve ratings'e atar
ratings = pd.DataFrame(df.groupby('title') ['rating'].mean())

#ardından ratingleri büyükten küçüğe sıralıyoruz
ratings.sort_values('rating',ascending = False).head(20)

,rating
title,
They Made Me a Criminal (1939),5.000000
Marlene Dietrich: Shadow and Light (1996),5.000000
"Saint of Fort Washington, The (1993)",5.000000
Someone Else's America (1995),5.000000
Star Kid (1997),5.000000
"Great Day in Harlem, A (1994)",5.000000
Aiqing wansui (1994),5.000000
Santa with Muscles (1996),5.000000
Prefontaine (1997),5.000000


## Bu ortalamalar hesaplanırken düşük oyu olan filmlere dikkat edilmediği için bilinmedik filmler ile karşılaştık.


In [16]:
#ratings isimli df'de 'oy_sayisi' isimli bir sütun oluştur ve title ile gruplanmış dataframden rating sayılarını al
ratings['oy_sayisi'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings

,rating,oy_sayisi
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41
...,...,...
Young Guns II (1990),2.772727,44
"Young Poisoner's Handbook, The (1995)",3.341463,41
Zeus and Roxanne (1997),2.166667,6


## en çok oy alan filmleri sıralayalım

In [17]:
ratings.sort_values('oy_sayisi',ascending=False)
#ascending true durumu küçükten büyüğe sıralar

,rating,oy_sayisi
title,,
Star Wars (1977),4.359589,584
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485
...,...,...
"Great Day in Harlem, A (1994)",5.000000,1
"Other Voices, Other Rooms (1997)",3.000000,1
Good Morning (1971),1.000000,1


In [18]:
corrStarwars = corrStarwars.join(ratings['oy_sayisi'])
corrStarwars.head(20)

,Correlation,oy_sayisi
title,,
'Til There Was You (1997),0.872872,9
1-900 (1994),-0.645497,5
101 Dalmatians (1996),0.211132,109
12 Angry Men (1957),0.184289,125
187 (1997),0.027398,41
2 Days in the Valley (1996),0.066654,93
"20,000 Leagues Under the Sea (1954)",0.289768,72
2001: A Space Odyssey (1968),0.230884,259
"39 Steps, The (1935)",0.106453,59


In [20]:
corrStarwars[corrStarwars['oy_sayisi']>100].sort_values('Correlation',ascending=False).head(10)

,Correlation,oy_sayisi
title,,
Star Wars (1977),1.000000,584
"Empire Strikes Back, The (1980)",0.748353,368
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Austin Powers: International Man of Mystery (1997),0.377433,130
"Sting, The (1973)",0.367538,241
Indiana Jones and the Last Crusade (1989),0.350107,331
Pinocchio (1940),0.347868,101
"Frighteners, The (1996)",0.332729,115
